In [1]:
import pandas as pd
import numpy as np

In [2]:
file = pd.read_csv('star_classification.csv')
file.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [3]:
file.drop('obj_ID', axis=1, inplace=True)
# file.drop('Event', axis=1, inplace=True)
file.dropna(inplace=True)
file.head()
print(file.__len__())

100000


In [4]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
file['class']= label_encoder.fit_transform(file['class'])

target = "class"
features = [col for col in file.columns if col != target]

In [5]:
file.head()

,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,0,0.634794,5812,56354,171
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,0,0.779136,10445,58158,427
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,0,0.644195,4576,55592,299
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,0,0.932346,9149,58039,775
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,0,0.116123,6121,56187,842


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(file[features], file[target], test_size=0.5, random_state=54)

In [7]:
from gplearn.genetic import SymbolicClassifier, SymbolicRegressor, SymbolicTransformer

sr = SymbolicRegressor(verbose=1, function_set={'add', 'sub', 'mul', 'div', 'sqrt', 'log', 'abs', 'neg', 'sin', 'cos', 'tan', 'inv', 'max', 'min'},parsimony_coefficient=0.0001, feature_names=file[features].columns)
sr.fit(X_train, y_train)
y_pred = sr.predict(X_test)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.19      6.56729e+20       20         0.420325              N/A     49.59s
   1     8.51      4.67702e+17        5         0.417698              N/A     43.35s
   2     4.22      1.31334e+18        6         0.306085              N/A     24.55s
   3     9.60      1.42117e+17        8         0.303412              N/A     30.48s
   4    11.80      3.35272e+17       13         0.294856              N/A     36.10s
   5    10.34      5.78469e+15       13         0.294856              N/A     35.81s
   6    12.64      5.78469e+15       23         0.222845              N/A     38.46s
   7    16.45      1.15694e+16       32         0.220596              N/A     39.57s
   8    20.16      1.73541e+16       27         0.196067              N/A  

In [8]:
from sklearn.metrics import accuracy_score
y_pred = [np.round(x) for x in y_pred]
accuracy_score(y_test, y_pred)

0.94448

In [9]:
import csv
with open('gplearn_class.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow([accuracy_score(y_test, y_pred)])

In [10]:
import pickle
pickle.dump(sr, open('model_class10.pkl', 'wb'))

In [11]:
print(sr._program)

tan(max(sqrt(sqrt(max(tan(max(div(inv(plate), neg(div(redshift, u))), inv(spec_obj_ID))), sin(tan(max(sqrt(sqrt(max(div(inv(plate), neg(redshift)), tan(sqrt(max(div(inv(plate), neg(redshift)), log(sin(max(tan(tan(neg(redshift))), max(div(inv(plate), neg(redshift)), inv(spec_obj_ID))))))))))), inv(inv(spec_obj_ID)))))))), sin(max(sin(sqrt(inv(sin(sqrt(sqrt(sqrt(sqrt(max(tan(max(div(inv(plate), neg(div(redshift, u))), inv(spec_obj_ID))), sin(tan(max(sqrt(sqrt(max(div(inv(plate), neg(redshift)), tan(sqrt(max(div(inv(plate), neg(redshift)), tan(sqrt(max(div(inv(plate), neg(redshift)), log(sin(max(tan(neg(redshift)), max(div(g, neg(redshift)), tan(sqrt(max(div(inv(plate), neg(redshift)), log(sin(max(tan(neg(redshift)), max(div(neg(plate), sin(redshift)), sin(tan(max(sqrt(sqrt(max(div(inv(plate), log(redshift)), tan(sqrt(max(div(inv(plate), neg(redshift)), log(sin(max(tan(neg(redshift)), inv(spec_obj_ID)))))))))), inv(inv(spec_obj_ID))))))))))))))))))))))))), inv(inv(spec_obj_ID)))))))))))))

In [1]:
import pickle

model = pickle.load(open('Classification\model_class7.pkl','rb'))
print(model._program)

div(tan(inv(div(div(div(tan(inv(spec_obj_ID)), log(log(g))), div(redshift, log(u))), div(div(log(g), add(g, MJD)), div(redshift, g))))), div(div(log(div(u, log(log(log(log(g)))))), add(g, MJD)), div(redshift, g)))
